In [ ]:
import numpy as np
import pickle
import json
import ast

file_paths = [
    './data/nasbench201/pickle/desktop-cpu-core-i7-7820x-fp32.pickle',
    './data/nasbench201/pickle/desktop-gpu-gtx-1080ti-fp32.pickle',
    './data/nasbench201/pickle/desktop-gpu-gtx-1080ti-large.pickle',
    './data/nasbench201/pickle/embedded-gpu-jetson-nono-fp16.pickle',
    './data/nasbench201/pickle/embedded-tpu-edgetpu-large.pickle',
    './data/nasbench201/pickle/embeeded-tpu-edgetpu-int8.pickle',
    './data/nasbench201/pickle/mobile-cpu-snapdragon-450-contex-a53-int8.pickle',
    './data/nasbench201/pickle/mobile-cpu-snapdragon-675-kryo-460-int8.pickle',
    './data/nasbench201/pickle/mobile-cpu-snapdragon-855-kryo-485-int8.pickle',
    './data/nasbench201/pickle/mobile-dsp-snapdragon-675-hexagon-685-int8.pickle',
    './data/nasbench201/pickle/mobile-dsp-snapdragon-855-hexagon-690-int8.pickle',
    './data/nasbench201/pickle/mobile-gpu-snapdragon-450-adreno-506-int8.pickle',
    './data/nasbench201/pickle/mobile-gpu-snapdragon-675-adren0-612-int8.pickle',
    './data/nasbench201/pickle/mobile-gpu-snapdragon-855-adren0-640-int8.pickle'
]


key_map = {}
idx = 0
first = True
all_df = np.zeros((len(file_paths), 15284, 6))
all_y = np.zeros((len(file_paths), 15284, 1))
for i, file_path in enumerate(file_paths):
    if first:
        idx = 0
        try:
            with open(file_path,'rb') as file:
                loaded_data = pickle.load(file)
                for key, value in loaded_data.items():
                    if key not in key_map:
                        all_df[i, idx] = key
                        key_map[key] = idx
                        all_y[i, idx] = value
                        idx += 1
        except Exception as e:
                print(f"Error loading {file_path}: {e}")
        first = False
    else:
        try:
            with open(file_path,'rb') as file:
                loaded_data = pickle.load(file)
                for key, value in loaded_data.items():
                    all_df[i, key_map[key]] = key 
                    all_y[i, key_map[key]] = value
        except Exception as e:
            print(f"Error loading {file_path}: {e}")

import os
for i, file_path in enumerate(file_paths):
    x = all_df[i]
    y = all_y[i]
    data = np.concatenate((x, y), axis=1)
    os.makedirs('./data/nasbench201/pkl', exist_ok=True)
    output_file_path = f'./data/nasbench201/pkl/{file_path.split("/")[-1].replace(".pickle", ".pkl")}'

    with open(output_file_path,'wb') as output_file:
        pickle.dump(data,output_file)

In [11]:
from data_provider.data_center import DataModule
from run_train import get_experiment_name
from utils.exp_config import get_config
from utils.exp_logger import Logger
from utils.exp_metrics_plotter import MetricsPlotter
from utils.utils import set_settings
config = get_config('OurModelConfig')
config.dataset = '201_acc'
set_settings(config)

log_filename, exper_detail = get_experiment_name(config)
plotter = MetricsPlotter(log_filename, config)
log = Logger(log_filename, exper_detail, plotter, config)
datamodule = DataModule(config)

<module 'module.name' from '/home/rtx4090/code/python/current/LightNAS/configs/OurModelConfig.py'> OurModelConfig
✅ All __pycache__ folders removed
|2025-08-26 13:49:07| {
     'ablation': 0, 'ac_loss': True, 'bs': 16
     'classification': False, 'continue_train': False, 'd_model': 100
     'dataset': 201_acc, 'debug': False, 'decay': 0.0001
     'device': cuda, 'dst_dataset': data/nasbench201/pkl/desktop-gpu-gtx-1080ti-fp32.pkl, 'embed_type': nerf
     'epochs': 600, 'eval_set': True, 'f': /run/user/1000/jupyter/runtime/kernel-v3adaa5158a9a8a049120fe198b64322efdce975d3.json
     'ffn_method': ffn, 'gcn_method': gcn, 'hyper_search': False
     'input_size': 192, 'log': <utils.exp_logger.Logger object at 0x710a7b6e2390>, 'logger': zyx
     'loss_func': MSELoss, 'lr': 0.001, 'model': ours
     'monitor_metric': NMAE, 'multires_p': 32, 'multires_r': 32
     'multires_x': 32, 'norm_method': batch, 'num_heads': 4
     'num_layers': 3, 'optim': Adam, 'path': ./data
     'patience': 50, 'pre

100%|██████████| 13909/13909 [00:03<00:00, 3897.50it/s]

|2025-08-26 13:49:11| Train_length : 764 Valid_length : 611 Test_length : 13909


In [12]:
data = datamodule.train_data 
data

{'key': [[0, 2, 1, 0, 4, 3],
  [3, 1, 1, 3, 3, 4],
  [1, 4, 1, 3, 2, 4],
  [4, 4, 3, 3, 3, 2],
  [4, 3, 3, 1, 0, 0],
  [1, 4, 2, 1, 0, 0],
  [4, 3, 2, 3, 3, 1],
  [3, 1, 1, 2, 3, 4],
  [3, 4, 3, 3, 2, 4],
  [1, 0, 3, 0, 3, 0],
  [1, 3, 1, 3, 0, 3],
  [0, 3, 0, 2, 4, 0],
  [1, 1, 3, 2, 0, 1],
  [2, 4, 2, 0, 1, 1],
  [2, 4, 0, 1, 3, 2],
  [3, 3, 3, 4, 1, 1],
  [0, 2, 2, 3, 2, 1],
  [3, 3, 0, 0, 0, 1],
  [3, 1, 2, 0, 1, 2],
  [4, 0, 0, 0, 3, 2],
  [4, 0, 0, 2, 3, 2],
  [4, 2, 1, 3, 0, 3],
  [2, 2, 3, 2, 3, 0],
  [3, 4, 4, 3, 2, 2],
  [2, 4, 0, 1, 3, 1],
  [4, 3, 2, 4, 1, 1],
  [3, 2, 3, 2, 1, 0],
  [2, 4, 4, 1, 0, 2],
  [0, 3, 1, 1, 1, 0],
  [3, 2, 3, 0, 3, 0],
  [3, 3, 3, 2, 3, 1],
  [1, 4, 1, 0, 2, 2],
  [1, 3, 0, 1, 2, 1],
  [1, 0, 3, 0, 2, 1],
  [4, 4, 1, 0, 1, 0],
  [1, 1, 0, 4, 2, 2],
  [1, 0, 1, 4, 3, 2],
  [1, 2, 4, 0, 4, 0],
  [4, 4, 1, 4, 4, 4],
  [3, 3, 3, 3, 3, 0],
  [2, 0, 1, 3, 0, 4],
  [3, 4, 0, 2, 3, 3],
  [2, 2, 1, 1, 4, 2],
  [1, 3, 3, 4, 1, 2],
  [1, 0, 1, 2, 3, 0],
  [

In [18]:
from baselines.narformer import tokenizer
from data_process.create_latency import get_arch_str_from_arch_vector, info2mat
idx = 0
key = data['key'][idx]
arch_str = get_arch_str_from_arch_vector(key)             # 架构向量转字符串
adj_mat, ops_idx = info2mat(arch_str)                     # 得到邻接矩阵与操作
tokens = tokenizer(ops_idx, adj_mat, 32, 32, 32, 'nerf')  # 得到token表示
N = len(ops_idx)

In [ ]:
len(ops_idx)

(8,)

In [2]:

import torch
import torch.nn as nn
import torch.nn.functional as F
class GraphIdentifierInjector(nn.Module):
    """
    为图数据添加各种结构标识符的工具类
    支持: rand_node_id, orf_node_id, lap_node_id, type_id
    """
    def __init__(
        self,
        node_feat_dim,          # 节点特征维度
        d_model,                # 最终特征维度
        rand_node_id=False,     # 是否使用随机节点ID
        rand_node_id_dim=64,    # 随机节点ID维度
        orf_node_id=True,      # 是否使用正交随机节点ID
        orf_node_id_dim=64,     # 正交随机节点ID维度
        lap_node_id=False,      # 是否使用拉普拉斯特征向量ID
        lap_node_id_k=8,        # 拉普拉斯特征向量数量
        lap_sign_flip=False,    # 拉普拉斯特征向量随机符号翻转
        type_id=False           # 是否使用类型ID（区分节点/边）
    ):
        super().__init__()
        self.node_feat_dim = node_feat_dim
        self.d_model = d_model
        self.lap_sign_flip = lap_sign_flip
        
        # 标识符开关
        self.rand_node_id = rand_node_id
        self.orf_node_id = orf_node_id
        self.lap_node_id = lap_node_id
        self.type_id = type_id
        
        # 计算输入特征总维度（原始特征 + 各类标识符）
        input_dim = node_feat_dim
        if lap_node_id:
            input_dim += lap_node_id_k
        if rand_node_id:
            self.rand_encoder = nn.Linear(2 * rand_node_id_dim, d_model, bias=False)
        if orf_node_id:
            self.orf_encoder = nn.Linear(2 * orf_node_id_dim, d_model, bias=False)
        if type_id:
            self.type_embed = nn.Embedding(2, d_model)  # 0:节点, 1:边
        
        # 节点特征投影层（融合原始特征与拉普拉斯ID）
        self.node_proj = nn.Linear(input_dim, d_model)
        
        # 拉普拉斯特征向量ID相关参数
        if lap_node_id:
            self.lap_node_id_k = lap_node_id_k

    def _compute_laplacian_feats(self, adj):
        """计算拉普拉斯特征向量标识符 [B, n, k]"""
        B, n, _ = adj.shape
        lap_feats = []
        for b in range(B):
            # 计算归一化拉普拉斯矩阵
            A = adj[b].float()
            deg = A.sum(1).clamp(min=1e-12)
            D_inv_sqrt = torch.diag(deg.pow(-0.5))
            L = torch.eye(n, device=adj.device) - D_inv_sqrt @ A @ D_inv_sqrt
            
            # 求解特征值和特征向量（取前k个最小特征值对应的特征向量）
            eigvals, eigvecs = torch.linalg.eigh(L)
            k = min(self.lap_node_id_k, n)
            lap_feat = eigvecs[:, :k]  # [n, k]
            
            # 填充到指定维度
            if k < self.lap_node_id_k:
                lap_feat = F.pad(lap_feat, (0, self.lap_node_id_k - k))
            
            lap_feats.append(lap_feat)
        
        lap_feats = torch.stack(lap_feats, dim=0)  # [B, n, k]
        
        # 随机符号翻转（增强鲁棒性）
        if self.lap_sign_flip and self.training:
            sign_flip = torch.randint(0, 2, lap_feats.shape[:2], device=adj.device) * 2 - 1  # [B, n]
            lap_feats = lap_feats * sign_flip.unsqueeze(-1)
        
        return lap_feats

    def _generate_rand_node_ids(self, node_num, device):
        """生成随机节点ID [sum(node_num), dim]"""
        total_nodes = sum(node_num)
        rand_ids = torch.randn(total_nodes, self.rand_node_id_dim, device=device)
        return F.normalize(rand_ids, p=2, dim=1)  # L2归一化保证正交性

    def _generate_orf_node_ids(self, node_num, max_n, device):
        """生成正交随机节点ID [sum(node_num), dim]"""
        B = len(node_num)
        # 生成高斯正交矩阵
        orf_matrix = self._gaussian_orthogonal_random_matrix(B, max_n, device)  # [B, max_n, max_n]
        
        # 提取有效节点的ID
        orf_ids = []
        ptr = 0
        for b in range(B):
            n = node_num[b]
            orf_ids.append(orf_matrix[b, :n, :n])  # [n, n]
            ptr += n
        
        orf_ids = torch.cat(orf_ids, dim=0)  # [sum(n), n]
        
        # 填充到指定维度
        if self.orf_node_id_dim > max_n:
            orf_ids = F.pad(orf_ids, (0, self.orf_node_id_dim - max_n), value=0.0)
        else:
            orf_ids = orf_ids[:, :self.orf_node_id_dim]
        
        return orf_ids

    @staticmethod
    def _gaussian_orthogonal_random_matrix(B, n, device):
        """生成批量化的高斯正交随机矩阵 [B, n, n]"""
        matrix = torch.randn(B, n, n, device=device)
        q, _ = torch.linalg.qr(matrix)  # QR分解获取正交矩阵
        return q

    def _get_node_mask(self, node_num, device):
        """生成节点掩码 [B, max_n]"""
        B = len(node_num)
        max_n = max(node_num)
        node_mask = torch.zeros(B, max_n, dtype=torch.bool, device=device)
        for b in range(B):
            node_mask[b, :node_num[b]] = True
        return node_mask

    def forward(self, adj, node_feats, edge_index=None):
        """
        为图数据添加各种标识符
        :param adj: 邻接矩阵 [B, n, n]
        :param node_feats: 节点特征 [B, n, c_node]
        :param edge_index: 边索引 [2, E] (可选，用于边标识符)
        :return: 
            node_tokens: 带标识符的节点token [B, n, d_model]
            edge_tokens: 带标识符的边token [B, e, d_model] (若提供edge_index)
        """
        B, n, _ = adj.shape
        device = adj.device
        node_num = [n] * B  # 假设批量中每个图的节点数都是n（可根据实际情况修改）
        node_mask = self._get_node_mask(node_num, device)
        
        # 1. 基础特征拼接（原始特征 + 拉普拉斯ID）
        input_feats = node_feats
        if self.lap_node_id:
            lap_feats = self._compute_laplacian_feats(adj)  # [B, n, k]
            input_feats = torch.cat([input_feats, lap_feats], dim=-1)  # [B, n, c_node + k]
        
        # 2. 节点投影（基础特征 -> d_model）
        node_tokens = self.node_proj(input_feats)  # [B, n, d_model]
        
        # 3. 添加随机节点ID标识符
        if self.rand_node_id:
            rand_ids = self._generate_rand_node_ids(node_num, device)  # [sum(n), dim]
            # 这里简化处理：直接将随机ID投影后加到节点特征（复杂场景可参考边索引关联逻辑）
            rand_embed = self.rand_encoder(torch.cat([rand_ids, rand_ids], dim=-1))  # [sum(n), d_model]
            ptr = 0
            for b in range(B):
                node_tokens[b, :node_num[b]] += rand_embed[ptr:ptr+node_num[b]]
                ptr += node_num[b]
        
        # 4. 添加正交随机节点ID标识符
        if self.orf_node_id:
            max_n = max(node_num)
            orf_ids = self._generate_orf_node_ids(node_num, max_n, device)  # [sum(n), dim]
            orf_embed = self.orf_encoder(torch.cat([orf_ids, orf_ids], dim=-1))  # [sum(n), d_model]
            ptr = 0
            for b in range(B):
                node_tokens[b, :node_num[b]] += orf_embed[ptr:ptr+node_num[b]]
                ptr += node_num[b]
        
        # 5. 添加类型ID（节点类型为0）
        if self.type_id:
            node_type = torch.zeros(n, dtype=torch.long, device=device)  # 节点类型标记为0
            node_tokens += self.type_embed(node_type).unsqueeze(0)  # [B, n, d_model]
        
        # 处理边标识符（如果提供edge_index）
        edge_tokens = None
        if edge_index is not None and (self.rand_node_id or self.orf_node_id or self.type_id):
            # 这里简化实现，实际可根据需要扩展
            edge_src, edge_dst = edge_index
            edge_tokens = torch.zeros(len(edge_src), self.d_model, device=device)
            
            # 边类型ID（类型为1）
            if self.type_id:
                edge_type = torch.ones(len(edge_src), dtype=torch.long, device=device)
                edge_tokens += self.type_embed(edge_type)
        
        return node_tokens, edge_tokens

# ====== 模拟一个 batch ======
B = 2       # batch size
N = 5       # 每个图 5 个节点
C = 16      # 原始节点特征维度

# 1. 构造邻接矩阵 (无向简单图)
adj = torch.zeros(B, N, N)
for b in range(B):
    for i in range(N-1):
        adj[b, i, i+1] = 1
        adj[b, i+1, i] = 1   # 对称化

# 2. 构造节点特征
node_feats = torch.randn(B, N, C)

# 3. 构造边索引 (示例：取第0个图的边)
edge_index = torch.tensor([
    [0, 1, 2],
    [1, 2, 3]
])  # [2, E]，表示 (0->1, 1->2, 2->3)

# ====== 使用 GraphIdentifierInjector ======
injector = GraphIdentifierInjector(
    node_feat_dim=C,
    d_model=32,           # 输出 token 维度
    rand_node_id=True,
    rand_node_id_dim=8,
    orf_node_id=True,
    orf_node_id_dim=8,
    lap_node_id=True,
    lap_node_id_k=4,
    lap_sign_flip=True,
    type_id=True
)

node_tokens, edge_tokens = injector(adj, node_feats, edge_index)

print("邻接矩阵 adj:", adj.shape)
print("节点特征 node_feats:", node_feats.shape)
print("输出 node_tokens:", node_tokens.shape)
print("输出 edge_tokens:", edge_tokens.shape if edge_tokens is not None else None)

AttributeError: 'GraphIdentifierInjector' object has no attribute 'rand_node_id_dim'

In [1]:
# Experiment Settings, logger, plotter
from exp.exp_model import Model
import utils
from utils.exp_logger import Logger
from utils.exp_metrics_plotter import MetricsPlotter
from utils.utils import set_settings
from utils.exp_config import get_config
from run_train import get_experiment_name
from data_provider.data_center import DataModule

config = get_config('OurModelConfig')
# config = get_config('FlopsConfig')
# config = get_config('MacConfig')
# config = get_config('LSTMConfig')
# config = get_config('GRUConfig')
# config = get_config('BRPNASConfig')
# config = get_config('GATConfig')
# config = get_config('NarFormerConfig')
# config = get_config('NarFormer2Config')
# config = get_config('NNformerConfig')
set_settings(config)

if config.dataset == 'nnlqp':
    config.input_size = 29
    if config.model == 'narformer':
        config.input_size = 1216
        config.graph_d_model = 960
        config.d_model = 1216
elif config.dataset == '101_acc':
    config.bs = 1024
        
log_filename, exper_detail = get_experiment_name(config)
plotter = MetricsPlotter(log_filename, config)
log = Logger(log_filename, exper_detail, plotter, config)

runid = 0
config.runid = runid
utils.utils.set_seed(config.seed + runid)
datamodule = DataModule(config)
model = Model(config)

/home/rtx4090/anaconda3/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


<module 'module.name' from '/home/rtx4090/code/python/current/LightNAS/configs/OurModelConfig.py'> OurModelConfig
✅ All __pycache__ folders removed
|2025-09-05 15:30:48| {
     'ablation': 0, 'ac_loss': True, 'bs': 256
     'classification': False, 'continue_train': False, 'd_model': 192
     'dataset': 201_acc, 'debug': False, 'decay': 0.0001
     'device': cuda, 'epochs': 1500, 'eval_set': True
     'f': /run/user/1000/jupyter/runtime/kernel-v31338a34f36b5bba3f820c9698ddb582b37dcfd12.json, 'hyper_search': False, 'input_size': 192
     'log': <utils.exp_logger.Logger object at 0x755ebf94da90>, 'logger': zyx, 'loss_func': MSELoss
     'lp_d_model': 16, 'lr': 0.001, 'model': ours
     'monitor_metric': KendallTau, 'monitor_reverse': True, 'num_heads': 4
     'num_layers': 2, 'op_encoder': embedding, 'optim': Adam
     'path': ./data, 'patience': 150, 'predict_target': accuracy
     'rank_loss': True, 'record': True, 'retrain': True
     'rounds': 5, 'runid': 0, 'scaler_method': minmax
 

In [2]:
one_batch = next(iter(datamodule.train_loader))

In [3]:
len(one_batch)

4

In [4]:
adj, features, P, y = one_batch
adj.shape

torch.Size([256, 8, 8])

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
torch.manual_seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"

# ---- soft Spearman（可微近似）----
def soft_rank(v, tau=1.0):
    v = v.view(-1,1)
    P = torch.sigmoid((v.T - v)/tau)   # P[j,i]≈I(v_j>v_i)
    r = 1.0 + P.sum(dim=1)
    return (r - r.mean()) / (r.std() + 1e-8)

def spearman_loss(a, b, tau=1.0):      # 越小越好
    ra, rb = soft_rank(a, tau), soft_rank(b, tau)
    rho = (ra * rb).mean()
    return 1.0 - rho

# ---- 代理（surrogate loss）网络：输入拼 (y_hat, y)，输出标量 L_i ----
class ReLossNet(nn.Module):
    def __init__(self, hid=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, hid), nn.ELU(),
            nn.Linear(hid, hid), nn.ELU(),
            nn.Linear(hid, 1)
        )
    def forward(self, y_hat, y):        # [B],[B] -> [B]
        x = torch.stack([y_hat, y], dim=-1)
        return self.net(x).squeeze(-1)

# ---- A 阶段：一次性训练 ReLoss（与指标排序一致）----
def train_reloss_once(steps=800, B=512, tau=1.0, lr=1e-3):
    reloss = ReLossNet().to(device)
    opt = optim.Adam(reloss.parameters(), lr=lr)
    for t in range(steps):
        # 合成混合分布（可替换为“历史模型输出 + 随机”）
        y = torch.randn(B, device=device)
        y_hat = torch.where(torch.rand(B, device=device)<0.5, y + 0.05*torch.randn_like(y),
                            0.7*y + 0.5*torch.randn_like(y))
        # 目标指标：这里用 per-sample“好坏分数”示意，也可用 batch 指标打分
        metric = 1.0 - torch.clamp((y_hat-y).abs()/(y.abs()+1e-8), 0, 1)  # 越大越好
        L = reloss(y_hat, y)                           # 越小代表越好
        loss = spearman_loss(-L, metric, tau)          # -L 与 metric 同序
        opt.zero_grad(); loss.backward(); opt.step()
    for p in reloss.parameters(): p.requires_grad_(False)
    reloss.eval()
    return reloss

# ---- B 阶段：用“冻结的 ReLoss”+ MSE 训练预测模型 ----
class Predictor(nn.Module):
    def __init__(self, in_dim=4, emb=64):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(in_dim,64), nn.ReLU(), nn.Linear(64,1))
    def forward(self, x): return self.net(x).squeeze(-1)

def sample_xy(B=512):
    z = torch.randn(B,2, device=device)
    y = 1.5*z[:,0] - 0.7*z[:,1] + 0.2*torch.randn(B, device=device)
    x = torch.cat([z, z**2], dim=1)   # [B,4]
    return x, y

def train_predictor_with_reloss(reloss, epochs=8, iters=100, beta=0.1):
    model = Predictor().to(device)
    opt = optim.Adam(model.parameters(), lr=1e-3)
    for ep in range(epochs):
        for _ in range(iters):
            x, y = sample_xy()
            y_hat = model(x)
            mse = ((y_hat - y)**2).mean()
            L_sur = reloss(y_hat.detach(), y).mean()   # 代理固定；可不 detach 也行
            loss = mse + beta * L_sur
            opt.zero_grad(); loss.backward(); opt.step()
    return model

# 用法
reloss = train_reloss_once()              # 一次性训练代理
model  = train_predictor_with_reloss(reloss, beta=0.1)  # 之后像普通 loss 一样用

In [ ]:
import numpy as np
from scipy import stats

realVec = np.array([0.2, 0.5, 0.3, 0.2])   # metric 向量 (真实 per-sample 分数)
estiVec = np.array([0.1, 0.8, 0.4, 0.2])   # ReLoss 输出向量 (预测的 loss)

kendall_tau = stats.kendalltau(realVec, estiVec).correlation
spearman_rho = stats.spearmanr(realVec, estiVec).correlation

print("Kendall τ:", kendall_tau)
print("Spearman ρ:", spearman_rho)

Kendall τ: 0.912870929175277
Spearman ρ: 0.9486832980505139


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# ============= 1) RankCosine Loss =============
class RankCosineLoss(nn.Module):
    """
    RankCosine (Qin et al., 2007)
    preds:  [B, N]  预测分数（越大越相关）
    labels: [B, N]  真实分数/等级（可用 0/1/2/3，或先做映射再传入）
    """
    def __init__(self, reduction: str = 'mean'):
        super().__init__()
        self.reduction = reduction

    def forward(self, preds: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
        preds, labels = preds.T, labels.T  # [B, 1]
        if preds.shape != labels.shape or preds.dim() != 2:
            raise ValueError("preds/labels must be [B, N] with same shape.")
        # 余弦相似度：越大越好 → 损失定义为 0.5*(1 - cos)
        cos = F.cosine_similarity(preds, labels, dim=1)          # [B]
        loss = 0.5 * (1.0 - cos)                                  # [B]
        if self.reduction == 'mean':
            return loss.mean()
        if self.reduction == 'sum':
            return loss.sum()
        return loss  # 'none'


# ============= 2) ListNet (Top-1) Loss =============
class ListNetTop1Loss(nn.Module):
    """
    ListNet (Top-1 Approximation) (Cao et al., 2007)
    用 Top-1 概率分布来做交叉熵：softmax(labels) vs softmax(preds)
    preds:  [B, N]
    labels: [B, N]  真实等级（数值越大越相关；可为 0/1/2/3）
    """
    def __init__(self, reduction: str = 'mean'):
        super().__init__()
        self.reduction = reduction

    def forward(self, preds: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
        preds, labels = preds.T, labels.T  # [B, 1]
        
        if preds.shape != labels.shape or preds.dim() != 2:
            raise ValueError("preds/labels must be [B, N] with same shape.")

        # 真实分布 P(y) 与 预测分布 P(s)
        p_y = torch.softmax(labels, dim=1)                        # [B, N]
        log_p_s = torch.log_softmax(preds, dim=1)                 # [B, N]

        # 交叉熵:  -sum_i P(y_i) * log P(s_i)
        loss = -(p_y * log_p_s).sum(dim=1)                        # [B]
        if self.reduction == 'mean':
            return loss.mean()
        if self.reduction == 'sum':
            return loss.sum()
        return loss  # 'none'
    
    

class ListMLELoss(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, preds, targets):
        # preds, targets: [N, 1] -> squeeze to [N]
        preds = preds.squeeze(-1)
        targets = targets.squeeze(-1)
        # 根据真实标签排序 (降序)
        order = torch.argsort(targets, descending=True)
        s_sorted = preds[order]  # [N]
        # 逐步计算 logsumexp
        rev = torch.flip(s_sorted, dims=[0])
        denom = torch.logcumsumexp(rev, dim=0)
        denom = torch.flip(denom, dims=[0])
        loss = (denom - s_sorted).sum()
        return loss

# ==== demo ====
torch.manual_seed(0)

preds = torch.tensor([[0.2, 0.8, 1.5, 0.3, 0.1]], dtype=torch.float32).T
targets = torch.tensor([[0.2, 0.8, 1.5, 0.3, 0.1]], dtype=torch.float32).T

print(preds.shape, targets.shape)
rc_crit  = RankCosineLoss()
ln_crit  = ListNetTop1Loss()
criterion = ListMLELoss()

print("RankCosine loss:", rc_crit(preds, targets).item())
print("ListNetTop1 loss:", ln_crit(preds, targets).item())
print("ListMLE loss:", criterion(preds, targets).item())


torch.Size([5, 1]) torch.Size([5, 1])
RankCosine loss: 2.9802322387695312e-08
ListNetTop1 loss: 1.4481860399246216
ListMLE loss: 3.461859703063965


In [ ]:
import torch

def diffkendall_tau(x, y, alpha=10.0, jitter=0.0):
    x = x.view(-1)
    y = y.view(-1)
    if jitter > 0:
        x = x + jitter * torch.randn_like(x)
        y = y + jitter * torch.randn_like(y)
    n = x.numel()

    dx = x.unsqueeze(0) - x.unsqueeze(1)   # [n,n]
    dy = y.unsqueeze(0) - y.unsqueeze(1)

    sx = torch.tanh(alpha * dx)
    sy = torch.tanh(alpha * dy)

    mask = torch.triu(torch.ones(n, n, device=x.device, dtype=torch.bool), diagonal=1)
    concord = (sx * sy)[mask].sum()

    num_pairs = torch.tensor(n * (n - 1) / 2, device=x.device, dtype=concord.dtype)
    return concord / num_pairs

def diffkendall_loss(pred, target, alpha=10.0, jitter=0.0):
    return 1.0 - diffkendall_tau(pred, target, alpha, jitter=jitter)

def soft_rank(v, tau=1.0):
    v = v.view(-1, 1)
    P = torch.sigmoid((v.T - v) / tau)     # [n,n]
    r = 1.0 + P.sum(dim=1)                 # 近似秩
    return (r - r.mean()) / (r.std() + 1e-8)

def spearman_loss(a, b, tau=1.0):
    ra, rb = soft_rank(a, tau), soft_rank(b, tau)
    rho = (ra * rb).mean()
    return 1.0 - rho

# demo
pred = torch.tensor([0.1, 0.4, 0.3, 0.9], requires_grad=True)
true = torch.tensor([1.0, 2.0, 3.0, 4.0])

loss_mse = torch.nn.functional.mse_loss(pred, true)
loss_spearman = spearman_loss(pred, true, tau=1.0)
loss_kendall = diffkendall_loss(pred, true, alpha=10.0)
print("MSE Loss:", loss_mse.item())
print("Spearman Loss:", loss_spearman.item())
print("DiffKendall Loss:", loss_kendall.item())
loss = loss_mse + loss_spearman + loss_kendall
loss.backward()

MSE Loss: 5.067500114440918
Spearman Loss: 0.35287100076675415
DiffKendall Loss: 0.3004358410835266


In [3]:
import torch
class SoftRankLoss(torch.nn.Module):
    def __init__(self, margin=0.0):
        super(SoftRankLoss, self).__init__()
        self.margin = margin
        
    def diffkendall_tau(self, x, y, alpha=10.0, jitter=0.0):
        x = x.view(-1)
        y = y.view(-1)
        if jitter > 0:
            x = x + jitter * torch.randn_like(x)
            y = y + jitter * torch.randn_like(y)
        n = x.numel()

        dx = x.unsqueeze(0) - x.unsqueeze(1)   # [n,n]
        dy = y.unsqueeze(0) - y.unsqueeze(1)

        sx = torch.tanh(alpha * dx)
        sy = torch.tanh(alpha * dy)

        mask = torch.triu(torch.ones(n, n, device=x.device, dtype=torch.bool), diagonal=1)
        concord = (sx * sy)[mask].sum()

        num_pairs = torch.tensor(n * (n - 1) / 2, device=x.device, dtype=concord.dtype)
        return concord / num_pairs

    def diffkendall_loss(self, pred, target, alpha=10.0, jitter=0.0):
        return 1.0 - self.diffkendall_tau(pred, target, alpha, jitter=jitter)

    def soft_rank(self, v, tau=1.0):
        v = v.view(-1, 1)
        P = torch.sigmoid((v.T - v) / tau)     # [n,n]
        r = 1.0 + P.sum(dim=1)                 # 近似秩
        return (r - r.mean()) / (r.std() + 1e-8)

    def spearman_loss(self, a, b, tau=1.0):
        ra, rb = self.soft_rank(a, tau), self.soft_rank(b, tau)
        rho = (ra * rb).mean()
        return 1.0 - rho

    def forward(self, preds, labels):
        """
        :param preds: Tensor of shape (B,) or (B, 1)
        :param labels: Tensor of shape (B,) or (B, 1)
        :return: Rank loss
        """
        # 自动 squeeze 支持 [B, 1] 输入
        if preds.ndim == 2 and preds.shape[1] == 1:
            preds = preds.squeeze(1)
        if labels.ndim == 2 and labels.shape[1] == 1:
            labels = labels.squeeze(1)

        if preds.ndim != 1 or labels.ndim != 1:
            raise ValueError("Both preds and labels must be 1D tensors.")

        loss_spearman = self.spearman_loss(preds, labels, tau=1.0)
        loss_kendall = self.diffkendall_loss(preds, labels, alpha=10.0)

        return loss_spearman, loss_kendall
    
if __name__ == "__main__":
    preds = torch.tensor([3.0, 1.0, 2.0, 4.0]).reshape(-1, 1)
    labels = torch.tensor([2.0, 1.0, 4.0, 3.0]).reshape(-1, 1)
    loss_fn = SoftRankLoss()
    loss_spearman, loss_kendall = loss_fn(preds, labels)
    print(f"Spearman Loss: {loss_spearman.item():.4f}, Kendall Loss: {loss_kendall.item():.4f}")

Spearman Loss: 0.7101, Kendall Loss: 0.6667


In [15]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from collections import defaultdict
from threading import Lock
import torch
from tqdm import tqdm  # 导入进度条库

def build_length_map_parallel(data_source, maxlen=None):
    """
    并行构建数据长度映射表，带进度条
    
    参数:
        data_source: 存储张量的列表，每个元素是形状为 (length, ...) 的张量
        maxlen: 最大长度限制，超过此长度的样本会被丢弃
    返回:
        长度到样本索引的映射字典
    """
    length_map = defaultdict(list)
    lock = Lock()  # 线程锁保证对length_map的安全操作
    
    def process_item(i):
        """处理单个数据项的函数"""
        try:
            data = data_source[i]
            length = len(data)  # 获取序列长度（第一维）
            
            if maxlen is not None and maxlen > 0 and length > maxlen:
                return None
            
            return (length, i)
        except Exception as e:
            print(f"处理索引{i}时出错: {e}")
            return None
    
    total = len(data_source)
    if total == 0:
        return dict(length_map)
    
    max_workers = 8
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_item, i) for i in range(total)]
        
        # 添加进度条，总任务数为total
        with tqdm(total=total, desc="处理进度") as pbar:
            for future in as_completed(futures):
                result = future.result()
                if result is not None:
                    length, idx = result
                    with lock:
                        length_map[length].append(idx)
                pbar.update(1)  # 每完成一个任务，进度条+1
    
    return dict(length_map)

# 生成测试数据（数量较多时更能体现进度条效果）
data_source = [torch.randn(torch.randint(1, 11, (1,)).item(), 3) for _ in range(385499)]

# 构建长度映射
length_map = build_length_map_parallel(data_source)

# 简要打印结果统计
print(f"\n完成！共处理 {len(data_source)} 个样本，映射到 {len(length_map)} 种长度")
# 如需详细打印可取消下面注释
# for length, indices in sorted(length_map.items()):
#     print(f"长度{length}: 共{len(indices)}个样本")

处理进度: 100%|██████████| 385499/385499 [00:00<00:00, 667007.68it/s]



完成！共处理 385499 个样本，映射到 10 种长度
